## Objectives

The goal for this project is to simulate the motion of a pendulum using Python.

If possible we would also like to add a driving force as well as damping to the simulation.



## Pendulum

Using two equations, $\tau= I\alpha$ and $\tau= -L(mgsin\theta)$, we can calculate the expected behaviour of a pendulum.

- $\tau$ is the torque \(in N m\).
- $I$ is the moment of inertia \(in kg m²\).
- $\alpha$ is the angular acceleration \(in rad s⁻²\).
- $L$ is the length of the string \(in m\).
- $g$ is the acceleration due to gravity \(in m s⁻²\)
- $\theta$ is the angular displacement \(in rad\).  

Combining the above:

$$
\tau = -L(mgsin\theta)
$$

$$
I\alpha = -L(mgsin\theta)
$$

$$
I\frac{d^2\theta}{dt^2} = -L(mgsin\theta)
$$

$$
mL^2\frac{d^2\theta}{dt^2} = -L(mgsin\theta)
$$

$$
\frac{d^2\theta}{dt^2} = -\frac{g}{L}sin\theta
$$

(1)

In order to solve this problem, numerically we can rewrite the second order differential equation above as two single differential equations.

$$
\frac{d\theta}{dt} = \omega
$$

$$
\frac{d\omega}{dt} = -\frac{g}{L} sin\theta
$$

Multiplying through by the time interval $dt$, we have: 

$$
d\theta = \omega\, dt
$$

$$
d\omega = -\frac{g}{L}sin\theta\,dt
$$

Starting from angular displacement $\theta_0$ and angular velocity $v_0$ at time $t_0 = 0$, we can calculate the change in angular displacement and angular velocity in the time $dt$ and hence find the new position and velocity at time 

$t_1 = t_0 +dt = dt$ :

$$
\theta_1 = \theta_0 + d\theta = \theta_0 + \omega_0
$$

$$
\omega_1 = \omega_0 +  d\omega = \omega_0 - \frac{g}{L}sin\theta_0\,dt
$$

Repeating this process gives us a way of calculating the angular displacement and angular velocity at any time:

$$
\theta_i = \theta_{i-1} + \omega_{i-1}\,dt
$$

$$
\omega_i = \omega_{i-1} - \frac{g}{L} sin\theta_{i-1}\,dt
$$

This is the Euler method



## Runge\-Kutta method

This method is similar to the Euler method, with some improvements:

\(2\)![](.15112022.ipynb.upload/paste-0.5142788792426083)

- the derivatives are calculated at multiple points along $dx$ 
- An average gradient is calculated in order to determine $y(x)$ at the next time value
- The positions at which these derivatives are calculated can be optimised

This is the method used in <span style='font-family:courier new'>solve\_ivp</span>, which is <span style='font-family:courier new'>scipy</span>'s ODE solver\- this is how we have decided to solve our ODE's numerically.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import FFMpegWriter
from scipy.integrate import solve_ivp
from IPython.display import HTML, display
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
def sim_Pendulum(l = 1.0, g = 9.81, theta_Start = 0.1, omega_Start = 0.0, tStop = 10.0, tStep = 0.05, name = "pendulum"):
    '''
     Function to simulate the motion of a pendulum.
    
    Parameters
    ----------
    m : float
        Mass of the pendulum bob (kg).
    l : float
        Length of the pendulum rod (m).
    g : float
        Acceleration due to gravity (m/s^2).
    theta_Start : float
        Initial position of the pendulum (rad).
    omega_Start : float
        Initial velocity of the pendulum (rad/s).
    tStop : float
        Time at which the simulation stops (s).
    tStep : float
        Time step for the simulation (s).
    nSteps : float 
        Number of time steps
    name : str
        Name of the output video file for the animation.

    Returns
    -------
    None
    
    Plots
    -----
    1. Angular position and angular velocity vs. time
    2. Angular position vs. angular velocity
    3. Animation of the pendulum's motion
    '''
    #
    nSteps = int(tStop/tStep) + 1
    #
    # arrays for plotting
    t = np.linspace(0.0, tStop, nSteps)
    theta = np.zeros(nSteps)
    omega = np.zeros(nSteps)
    #
    # Initialise y
    yInit = [theta_Start, omega_Start]
    #
    def dy(t,y):
        '''
        dy/dt = f(t,y) where t is time and y = [theta, omega] where theta is angular displacement and omega is dTheta/dt.
        This function returns the slope in time for y in both elements:
        dy/dt = [d_theta/dt, d_omega/dt] = [omega, -g/l sin(theta)]
        '''
        #
        theta = y[0]
        omega = y[1]
        #
        dydt = np.zeros(2)
        dydt[0] = y[1]
        dydt[1] = (-g/l)*np.sin(y[0])
        #
        return dydt
    #
    tInterval = [0.0, tStop]
    #
    sol = solve_ivp(dy, tInterval, yInit, t_eval = t) #(Reference 3)
    #
    t = sol.t
    theta = sol.y[0]
    omega = sol.y[1]
    #
    # Create a figure with two subplots.
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the first graph in the first subplot.
    ax1.plot(t, theta, color = 'm', linestyle ='-', label = r'$\theta$')
    ax1.plot(t, omega, color = 'g', linestyle = '-', label = r'$\omega$')
    ax1.set_title("Angular Position and Angular Velocity of a pendulum")
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel(r'$\theta$ (rad), $\omega$ (rad/s)')
    ax1.legend()
    ax1.grid()

    # Plot the second graph in the second subplot.
    ax2.plot(theta, omega, color = 'm', linestyle ='-', label = r'$\theta$')
    ax2.set_title("Angular Position vs Angular Velocity of a pendulum")
    ax2.set_xlabel(r'$\theta$ (rad)')
    ax2.set_ylabel(r'$\omega$ (rad/s)')
    ax2.legend()
    ax2.grid()

    # Show the figure.
    plt.savefig('Graphs/{}.png'.format(name))
    plt.show()
    
    '''
    Animation of the simulated pendulum (Reference 5)
    '''
    def get_coords(theta):
        """Return the (x, y) coordinates of the bob at angle theta."""
        return l * np.sin(theta), -l * np.cos(theta)

    # Initialize the animation plot. Make the aspect ratio equal so it looks right.
    fig = plt.figure(figsize = [6,6])
    ax = fig.add_subplot(aspect='equal')
    #
    # The pendulum rod, in its initial position.
    x0, y0 = get_coords(theta_Start)
    line, = ax.plot([0, x0], [0, y0], lw=3, c='k')
    #
    # The pendulum bob: set zorder so that it is drawn over the pendulum rod.
    bob_radius = 0.08
    circle = ax.add_patch(plt.Circle(get_coords(theta_Start), bob_radius, fc='r', zorder=3))
    #
    # Set the plot limits so that the pendulum has room to swing!
    ax.set_xlim(-l*1.2, l*1.2)
    ax.set_ylim(-l*1.2, l*1.2)
    ax.tick_params(labelbottom=False, labelleft=False) # removing arbritrary tick labels

    def animate(i):
        """Update the animation at frame i."""
        x, y = get_coords(theta[i])
        line.set_data([0, x], [0, y])
        circle.set_center((x, y))

    nFrames = nSteps
    interval = tStep * len(t)
    anim = animation.FuncAnimation(fig, animate, frames=nFrames, repeat=True, interval=interval)
    #
    html = HTML(anim.to_html5_video())
    display(html) #(Reference 6)
    #
    writer = FFMpegWriter(fps=30, bitrate=1800)
    anim.save('Videos/{}.mp4'.format(name), writer = writer) #(Reference 7)

In [3]:
sim_Pendulum(l = 1.0, g = 9.81, theta_Start = 0.1, omega_Start = 0.0, tStop = 10.0, tStep = 0.05, name = "basic pendulum")


In [6]:
# Using the the interaction function to make interactive sliders for parameters
im = interact_manual(sim_Pendulum, l = (0.1, 5, 0.1),
                     g = (1.6, 25., 0.01),
                     theta_Start = (-1., 1., 0.1),
                     omega_Start = (-1., 1., 0.1),
                     tStop = (10, 1000, 10),
                     tStep = (0.01, 1.,0.01))

interactive(children=(FloatSlider(value=1.0, description='l', max=5.0, min=0.1), FloatSlider(value=9.81, descr…

## <span style='font-size:medium'>**Adding friction and a driving term**</span>

We will now add a term for friction and the driving force to the equation:

$$
\frac{d^2\theta}{dt^2} = -\frac{g}{L}sin\theta
$$

Here the variables are the same as defined above, with the addition of the following variables;

- $b =$ damping \(friction\) constant
- $\tau_0 =$ amplitude of driving force
- $c =$ constant related to frequency of driving force 

As defined above, the torque due to gravity is:

$$
\tau_g = -L(mgsin\theta)
$$

We now need to include the torque due to friction and the driving force, shown below, respectively;

$$
\tau_f = -b\omega
$$

$$
\tau_d=\tau_0cos(ct)
$$

Using the following equations:

$$
\tau= I\alpha
$$

$$
I=mL^2
$$

And the equations defined for torque, we can now rewrite the differential equation for a damped and driven pendulum as:

$$
\frac{d^2\theta}{dt^2} = -\frac{g}{L}sin\theta+\frac{\tau_0cos(ct)-bd\omega}{mL^2}
$$

\(4\)

Reqriting a two first order differential equations as above:

$$
\frac{d\theta}{dt} = \omega
$$

$$
\frac{d\omega}{dt} = \frac{\tau_0}{mL^2}cos(ct) - \frac{b}{mL^2}\omega - \frac{g}{L}sin(\theta)
$$

We can now see how we need to modify the above routines to add the new terms. We will use `solve_ivp`. 



In [7]:
def sim_Pendulum2(m = 1.0, l = 1.0, g = 9.81, b = 0.1, tau0 = 1.0, c = 1.0, 
               theta_Start = 0.1, omega_Start = 0.0, tStop = 10.0, tStep = 0.1, name = "pendulum", save = False):
    """
    Function to simulate the motion of a pendulum with damping and/or a driving force.
    
    Parameters
    ----------
    m : float
        Mass of the pendulum bob (kg).
    l : float
        Length of the pendulum rod (m).
    g : float
        Acceleration due to gravity (m/s^2).
    b : float
        Damping coefficient (kg/s).
    tau0 : float
        Magnitude of the driving force (Nm).
    c : float
        Angular frequency of the driving force (rad/s).
    theta_Start : float
        Initial position of the pendulum (rad).
    omega_Start : float
        Initial velocity of the pendulum (rad/s).
    tStop : float
        Time at which the simulation stops (s).
    tStep : float
        Time step for the simulation (s).
    name : str
        Name of the output video file for the animation.
    save : bool
        Toggles whether graphs and animations are saved.

    Returns
    -------
    None
    
    Plots
    -----
    1. Angular position and angular velocity vs. time
    2. Angular position vs. angular velocity
    3. Animation of the pendulum's motion
    """
    #
    nSteps = int(tStop/tStep) + 1
    #
    # arrays for plotting
    t = np.linspace(0.0, tStop, nSteps)
    theta = np.zeros(nSteps)
    omega = np.zeros(nSteps)
    #
    # Initialise y
    yInit = [theta_Start, omega_Start]
    #
    def dy(t,y):
        '''
        dy/dt = f(t,y) where t is time and y = [theta, omega] where theta is angular displacement and omega is dTheta/dt.
        This function returns the slope in time for y in both elements:
        dy/dt = [d_theta/dt, d_omega/dt] = [omega, -g/l sin(theta)]
        '''
        #
        theta = y[0]
        omega = y[1]
        #
        dydt = np.zeros(2)
        dydt[0] = y[1]
        dydt[1] = (tau0/m*l**2)*np.cos(c*t)-(b/m*l**2)*y[1]-(g/l)*np.sin(y[0])
        #
        return dydt
    #
    tInterval = [0.0, tStop]
    #
    sol = solve_ivp(dy, tInterval, yInit, t_eval = t) # Reference (3)
    #
    t = sol.t
    theta = sol.y[0]
    omega = sol.y[1]
    #
    # Create a figure with two subplots.
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the first graph in the first subplot.
    ax1.plot(t, theta, color = 'm', linestyle ='-', label = r'$\theta$')
    ax1.plot(t, omega, color = 'g', linestyle = '-', label = r'$\omega$')
    ax1.set_title("Angular Position and Angular Velocity of a pendulum")
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel(r'$\theta$ (rad), $\omega$ (rad/s)')
    ax1.legend()
    ax1.grid()

    # Plot the second graph in the second subplot.
    ax2.plot(theta, omega, color = 'm', linestyle ='-', label = r'$\theta$')
    ax2.set_title("Angular Position vs Angular Velocity of a pendulum")
    ax2.set_xlabel(r'$\theta$ (rad)')
    ax2.set_ylabel(r'$\omega$ (rad/s)')
    ax2.legend()
    ax2.grid()

    # Show the figure.
    if save:
        plt.savefig('Graphs/{}.png'.format(name))
    plt.show()
    
    '''
    Creating Animation of the simulated pendulum (Reference 5)
    '''
    def get_coords(theta):
        """Return the (x, y) coordinates of the bob at angle theta."""
        return l * np.sin(theta), -l * np.cos(theta)

    # Initialize the animation plot. Make the aspect ratio equal so it looks right.
    fig = plt.figure(figsize = [6,6])
    ax = fig.add_subplot(aspect='equal')
    #
    # The pendulum rod, in its initial position.
    x0, y0 = get_coords(theta_Start)
    line, = ax.plot([0, x0], [0, y0], lw=3, c='k')
    #
    # The pendulum bob: set zorder so that it is drawn over the pendulum rod.
    bob_radius = 0.08
    circle = ax.add_patch(plt.Circle(get_coords(theta_Start), bob_radius, fc='r', zorder=3))
    #
    # Set the plot limits so that the pendulum has room to swing!
    ax.set_xlim(-l*1.2, l*1.2)
    ax.set_ylim(-l*1.2, l*1.2)
    ax.tick_params(labelbottom=False, labelleft=False) # removing arbritrary tick labels
    plt.title("{}".format(name), fontsize = 16)
    #
    def animate(i):
        """Update the animation at frame i."""
        x, y = get_coords(theta[i])
        line.set_data([0, x], [0, y])
        circle.set_center((x, y))

    nFrames = nSteps
    interval = tStep * len(t)
    anim = animation.FuncAnimation(fig, animate, frames=nFrames, repeat=True, interval=interval)
    #
    html = HTML(anim.to_html5_video())
    display(html) # (Reference 6)
    #
    writer = FFMpegWriter(fps=30, bitrate=1800)
    if save:
        anim.save('Videos/{}.mp4'.format(name), writer = writer) # (Reference 7)

In [8]:
sim_Pendulum2(l = 1.0, g = 9.81, b = 0.1, tau0 = 1.0, c = 1.0,
               theta_Start = 0.1, omega_Start = 0.0, tStop = 20.0, tStep = 0.1, name = "driven and damped pendulum")

In [10]:
im2 = interact_manual(sim_Pendulum2, m = (0.1, 10, 0.1), 
                     l = (0.1, 5, 0.1),
                     g = (1.6, 25., 0.01),
                     b = (0, 1, 0.1),
                     tau0 = (0, 10, 0.1),
                     c = (0, 10, 0.01),
                     theta_Start = (-np.pi, np.pi, np.pi/16),
                     omega_Start = (-1., 1., 0.1),
                     tStop = (10, 1000, 10),
                     tStep = (0.01, 1.,0.01))

interactive(children=(FloatSlider(value=1.0, description='m', max=10.0, min=0.1), FloatSlider(value=1.0, descr…

### Resonance

Resonance is achieved when the frequency of the driving torque, $f_d$, is equal to natural frequency of oscillation of the pendulum, $f_n$.

The time period of a pendulum, $T$, is given by:

$$
T =  2\pi \sqrt(\frac{l}{g})
$$

$$
f_n = \frac{1}{T}
$$

$$
\Rightarrow f_n = \frac{1}{2\pi} \sqrt(\frac{g}{l})
$$

The driving torque, $\tau_d$, is given by:

$$
\tau_d = \tau_0 cos(ct)
$$

where $c = 2\pi f_d$,

setting $f_d = f_n$ we get:

$$
c = 2\pi f_n
$$

$$
\Rightarrow c = 2\pi \, \frac{1}{2\pi} \sqrt(\frac{g}{l})
$$

$$
c = \sqrt(\frac{g}{l})
$$



In [11]:
sim_Pendulum2(l = 1.0, g = 9.81, b = 0.1, tau0 = 0.5, c = 3.132091952673165, 
               theta_Start = 0.0, omega_Start = 0.0, tStop = 20.0, tStep = 0.1, name = "resonant driving torque pendulum")

### Critical damping

Critical damping occurs when $\omega$ is reduced to 0 without overshooting.


In [12]:
sim_Pendulum2(l = 1.0, g = 9.81, b = 5.5805, tau0 = 0, c = 0, 
               theta_Start = 0.3, omega_Start = 0.0, tStop = 20.0, tStep = 0.1, name = "critically damped pendulum")

## Conclusion

We successfully met our objectives by simulating the motion of a pendulum, with and without a driving force and damping. To see the simulation in action look at the 'Graphs' and 'Videos' folder in the Files of the project.

We also decided to simulate the motion of a pendulum in places other than on Earth \(e.g. the moon\) by changing the value of g.

However one unsolved issue we ran into was that the simulations/ videos produced a screenshot \(from the video\) at the bottom of the cell, which was unwanted. We attempted to remove this to no avail and we believe this could be a limitation of the Jupyter Notebook as nothing we tried worked. For example we attempted to use <span style='font-family:courier new'>clear\_output</span> from IPython however this didn't work, hence why it hasn't been included in the code. 

A possible extension to this project could be to produce a 3D animation, however we are unaware of a library that currently provides this, so it could be necessary to write this ourselves which would take a long time.


### References:

(1) [Derivation of Second Order Differential for a Pendulum](https://phys.libretexts.org/Bookshelves/University_Physics/Book%3A_University_Physics_(OpenStax)/Book%3A_University_Physics_I_-_Mechanics_Sound_Oscillations_and_Waves_(OpenStax)/15%3A_Oscillations/15.05%3A_Pendulums)

\(2\) [Runge\-Kutta Method](https://people.revoledu.com/kardi/tutorial/ODE/Runge%20Kutta%202.htm)

\(3\) [Solving differential equations from SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) 

\(4\) [Damped and Driven Oscillations of a Pendulum](https://www.myphysicslab.com/pendulum/chaotic-pendulum-en.html#:~:text=A%20damped%20driven%20pendulum%20is,a%20short%20period%20of%20time)

\(5\) [Matplotlib animation tutorial](https://scipython.com/book2/chapter-7-matplotlib/problems/p77/animating-a-pendulum/)

\(6\) [IPython documentation, Display video in a Juypter Notebook](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html)

\(7\) [Matplotlib documentation, Save the video of the simulation to a directory](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FFMpegWriter.html)
